In [2]:
import os

import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable as var

import torchvision
from torchvision import datasets, transforms, models

import collections

import json

import time
import datetime

import PIL.Image as img

from collections import OrderedDict

In [2]:
def fun(input, color=_):
    s = str(input)
    if color == 'red':
        s = '\033[91m' + s
    if color == 'green':
        s = '\033[92m' + s
    return s + '\033[0m'

def cmd(command):
    os.system(f'cmd /c \"{command}\"')

In [4]:
path = 'FlagNet_data'
token = '6k2wkUdrjTGbOzuWXfTVciCT0Rd3TV14urIH'
repo = f'https://ghp_{token}@github.com/yuval-ro/FlagNet_data.git'

cmd(f'git clone {repo} {path}')
cmd(f'del {path}\\.gitignore')
cmd(f'del {path}\\README.md')

In [ ]:
# Dataset sanity check and display:
IMAGES_PER_CLASS = 30
dataset_dir = 'repo\\dataset'
classes_json = 'repo\\classes.json'
images_per_class = []
bad_dirs = []
json_ne_dirs = False

# Locates and parses the "classes.json" file:
try:
    with open(classes_json, 'r') as f:
        classes = OrderedDict(json.load(f))
        dir_names = [dataset_dir + '\\%.2d' % i for i in range(1, len(classes) + 1)]
except FileNotFoundError:
    pass

# Checks the number of classes defined in the json equal to number of classes subdirs:
if len(os.listdir(dataset_dir)) != len(classes):
    json_ne_dirs=True

# Checks the number of images of each class subdir:
for dir_tuple in os.walk(dataset_dir):
    if dir_tuple[0] in dir_names: # skips junk directories
        images_in_dir = len(dir_tuple[2])
        images_per_class.append(images_in_dir)
        if images_in_dir != IMAGES_PER_CLASS:
            bad_dirs.append(dir_tuple[0])

# Displays the metadata nicely:
def println(s1='', s2='', s3='', bold=False):
    if bold:
        print('\033[1m', end='')
    print(  '{0:<20}'.format(s1),
            '{0:<20}'.format(s2),
            '{0:<20}'.format(s3))
    if bold:
        print('\033[0m', end='')

println('id', 'class', 'images', bold=True)

for i, (ID, Class) in enumerate(classes.items()):
    println(ID,
            Class.upper() if Class in ['uk','usa'] else Class.capitalize(),
            images_per_class[i] if images_per_class[i] == IMAGES_PER_CLASS else fun(images_per_class[i], color='red'))
println(s3='total', bold=True)
println(s3=(sum(images_per_class)) if sum(images_per_class) == (len(classes) * IMAGES_PER_CLASS) else fun(sum(images_per_class), color='red'))

# Throws exceptions if needed:
if json_ne_dirs:
    raise SystemExit(f'number of classes according to the json file ({len(classes)}) does not correlate with total dirs ({len(os.listdir(dataset_dir))}) in \"{dataset_dir}\"')
elif bad_dirs != []:
    raise SystemExit(f'image count in the following directories is incorrect: {bad_dirs}')
else:
    println(s3=fun('all okay!', color='green'), bold=True)